In [59]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from tqdm import tqdm
from scipy import sparse

In [2]:
weather_file = pd.read_csv("Data/weather_processed.csv", sep = ',')
#date와 station_nbr정보를 가져옴
multi_index = [weather_file.values[:,1], weather_file.values[:,0]]
#index를 date와 station_nbr기준으로 하여 데이터를 가져옴
df = pd.DataFrame(weather_file.values[:,2:], index=multi_index, columns=weather_file.columns.values[2:])
df

tmax tmin tavg depart dewpoint wetbulb heat cool sunrise sunset  \
2012-01-01 1    52   31   42     16       36      40   23    0     716   1626   
           2    48   33   41     16       37      39   24    0     716   1626   
           3    55   34   45      9       24      36   20    0     735   1720   
           4    63   47   55      4       28      43   10    0     728   1742   
           6    63   34   49      0       31      43   16    0     727   1742   
           7    50   33   42      0       26      35   23    0     727   1742   
           8    66   45   42      0       34      46   23    0     727   1742   
           9    34   19   27      0       17      23   38    0     727   1742   
           10   73   53   63      0       55      58    2    0     723   1738   
           11   72   48   60      7       54      56    5    0     724   1737   
           12   72   48   60      7       54      57    5    0     724   1737   
           13   38   18   28      7       12      21   37    0     724   1737   
           14   50   34   42      5       25      35   23    0     739   1729   
           15   48   26   37     16       35      38   28    0     718   1621   
           16   50   30   40     16       37      40   25    0     718   1621   
           17   56   38   47     16       23      37   18    0     718   1621   
           18   59   40   50      4       28      40   15    0     730   1732   
           19   38   25   32     10       26      30   33    0     723   1628   
           20   59   41   50     10       28      40   15    0     723   1628   
2012-01-02 1    50   31   41     10       26      35   24    0     723   1628   
           2    46   28   37     12       24      32   28    0     716   1627   
           3    45   24   35     -1       11      28   30    0     735   1721   
           4    60   37   49     -2       26      39   16    0     728   1742   
           6    60   30   45     -4       27      38   20    0     727   1742   
           7    44   26   35     -4       17      29   30    0     727   1742   
           8    59   36   35     -4       29      40   30    0     727   1742   
           9    21   11   16     -4        8      15   49    0     727   1742   
           10   65   45   55     -4       33      46   10    0     723   1738   
           11   63   40   52     -1       31      44   13    0     724   1738   
           12   65   47   56     -1       33      46    9    0     724   1738   
...            ...  ...  ...    ...      ...     ...  ...  ...     ...    ...   
2014-10-30 11   81   59   70      4       63      65    0    5     640   1741   
           12   81   61   71      4       63      66    0    6     640   1741   
           13   61   32   47      4       26      38   18    0     640   1741   
           14   77   45   61      5       39      50    4    0     650   1738   
           15   56   30   43      0       36      39   22    0     618   1641   
           16   57   34   46      0       39      43   19    0     618   1641   
           17   77   43   60      0       41      50    5    0     618   1641   
           18   79   56   68      5       51      58    0    3     643   1739   
           19   51   32   42     -4       33      40   23    0     623   1647   
           20   80   52   66     -4       50      57    0    1     623   1647   
2014-10-31 1    53   40   47     -4       36      41   18    0     623   1647   
           2    49   40   45      0       35      40   20    0     619   1643   
           3    55   33   44    -12       32      41   21    0     645   1729   
           4    76   56   66     -1       47      56    0    1     644   1745   
           5    76   56   66     -1       37      56    0    1     616   1640   
           6    76   51   64      0       47      55    1    0     644   1745   
           7    58   38   48      0       32      41   17    0     644   1745   
           8    78   55   67      0       32 

In [14]:
store_station_pairs = match_store_station()
store_station_pairs
# station_nbr = store_station_pairs.loc[store_nbr].values[0]
# weather_data = weather_file.loc[date, station_nbr].values

NameError: name 'match_store_station' is not defined

In [15]:
key_file = pd.read_csv("Data/key.csv", sep=',')
key_file

,store_nbr,station_nbr
0,1,1
1,2,14
2,3,7
3,4,9
4,5,12
5,6,14
6,7,6
7,8,4
8,9,17
9,10,12


In [16]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [17]:
weather_file = pd.read_csv("Data/weather.csv", sep=',')
weather_data = weather_file.drop('codesum', axis = 1) # codesum 컬럼 삭제
    
data = weather_data.values[:,2:] # index, station_nbr, date를 제외한 나머지 데이터 array로 변경
# 데이터가 있을 경우 float형으로 변경, 데이터가 없을 경우 unicodedata로 변경
for col in range(data.shape[1]):
    for row in range(data.shape[0]):
        if not is_number(data[row, col]):
            if row == 0:
                i = 1
                while not is_number(data[row+i, col]):
                    i += 1
                data[row, col] = data[row+i, col]
            else:
                data[row, col] = data[row-1, col]
        data[row, col] = float(data[row, col]) # Convert string to float value

weather_processed = np.append(weather_file.values[:,0:2], data, axis = 1)
df = pd.DataFrame(weather_processed, index=None, columns=weather_data.columns.values)
    
df

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,16,36,40,23,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,716,1626,0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,735,1720,0,0,29.77,30.47,9.9,31,10
3,4,2012-01-01,63,47,55,4,28,43,10,0,728,1742,0,0,29.79,30.48,8,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,727,1742,0,0,29.95,30.47,14,36,13.8
5,7,2012-01-01,50,33,42,0,26,35,23,0,727,1742,0,0,29.15,30.54,10.3,32,10.2
6,8,2012-01-01,66,45,42,0,34,46,23,0,727,1742,0,0,30.05,30.54,11,36,10.9
7,9,2012-01-01,34,19,27,0,17,23,38,0,727,1742,0,0,29.34,30.09,22.8,30,22.5
8,10,2012-01-01,73,53,63,0,55,58,2,0,723,1738,0,0,30.16,30.19,5.1,24,5.5
9,11,2012-01-01,72,48,60,7,54,56,5,0,724,1737,0,0,30.15,30.18,4.6,23,4.8


In [18]:
def process_weather():
    weather_file = pd.read_csv("Data/weather.csv", sep=',')
    weather_data = weather_file.drop('codesum', axis = 1) # codesum 컬럼 삭제
    
    data = weather_data.values[:,2:] # index, station_nbr, date를 제외한 나머지 데이터 array로 변경
    # 데이터가 있을 경우 float형으로 변경, 데이터가 없을 경우 unicodedata로 변경
    for col in range(data.shape[1]):
        for row in range(data.shape[0]):
            if not is_number(data[row, col]):
                if row == 0:
                    i = 1
                    while not is_number(data[row+i, col]):
                        i += 1
                    data[row, col] = data[row+i, col]
                else:
                    data[row, col] = data[row-1, col]
            data[row, col] = float(data[row, col]) # Convert string to float value

    weather_processed = np.append(weather_file.values[:,0:2], data, axis = 1)
    df = pd.DataFrame(weather_processed, index=None, columns=weather_data.columns.values)
    df.to_csv("Data/weather_processed.csv", sep=',', index=None)

In [19]:
#index를 제외한 key데이터를 dataframe로 가져옴
def match_store_station():
    key_file = pd.read_csv("Data/key.csv", sep=',')
    df = pd.DataFrame(key_file.values[:,1], index = key_file.values[:,0])
    return df

In [30]:
def read_processed_weather():
    weather_file = pd.read_csv("Data/weather_processed.csv", sep = ',')
    #date와 station_nbr정보를 가져옴
    multi_index = [weather_file.values[:,1], weather_file.values[:,0]]
    #index를 date와 station_nbr기준으로 하여 데이터를 가져옴
    df = pd.DataFrame(weather_file.values[:,2:], index=multi_index, columns=weather_file.columns.values[2:])
    return df

In [31]:
a = read_processed_weather()
a

tmax tmin tavg depart dewpoint wetbulb heat cool sunrise sunset  \
2012-01-01 1    52   31   42     16       36      40   23    0     716   1626   
           2    48   33   41     16       37      39   24    0     716   1626   
           3    55   34   45      9       24      36   20    0     735   1720   
           4    63   47   55      4       28      43   10    0     728   1742   
           6    63   34   49      0       31      43   16    0     727   1742   
           7    50   33   42      0       26      35   23    0     727   1742   
           8    66   45   42      0       34      46   23    0     727   1742   
           9    34   19   27      0       17      23   38    0     727   1742   
           10   73   53   63      0       55      58    2    0     723   1738   
           11   72   48   60      7       54      56    5    0     724   1737   
           12   72   48   60      7       54      57    5    0     724   1737   
           13   38   18   28      7       12      21   37    0     724   1737   
           14   50   34   42      5       25      35   23    0     739   1729   
           15   48   26   37     16       35      38   28    0     718   1621   
           16   50   30   40     16       37      40   25    0     718   1621   
           17   56   38   47     16       23      37   18    0     718   1621   
           18   59   40   50      4       28      40   15    0     730   1732   
           19   38   25   32     10       26      30   33    0     723   1628   
           20   59   41   50     10       28      40   15    0     723   1628   
2012-01-02 1    50   31   41     10       26      35   24    0     723   1628   
           2    46   28   37     12       24      32   28    0     716   1627   
           3    45   24   35     -1       11      28   30    0     735   1721   
           4    60   37   49     -2       26      39   16    0     728   1742   
           6    60   30   45     -4       27      38   20    0     727   1742   
           7    44   26   35     -4       17      29   30    0     727   1742   
           8    59   36   35     -4       29      40   30    0     727   1742   
           9    21   11   16     -4        8      15   49    0     727   1742   
           10   65   45   55     -4       33      46   10    0     723   1738   
           11   63   40   52     -1       31      44   13    0     724   1738   
           12   65   47   56     -1       33      46    9    0     724   1738   
...            ...  ...  ...    ...      ...     ...  ...  ...     ...    ...   
2014-10-30 11   81   59   70      4       63      65    0    5     640   1741   
           12   81   61   71      4       63      66    0    6     640   1741   
           13   61   32   47      4       26      38   18    0     640   1741   
           14   77   45   61      5       39      50    4    0     650   1738   
           15   56   30   43      0       36      39   22    0     618   1641   
           16   57   34   46      0       39      43   19    0     618   1641   
           17   77   43   60      0       41      50    5    0     618   1641   
           18   79   56   68      5       51      58    0    3     643   1739   
           19   51   32   42     -4       33      40   23    0     623   1647   
           20   80   52   66     -4       50      57    0    1     623   1647   
2014-10-31 1    53   40   47     -4       36      41   18    0     623   1647   
           2    49   40   45      0       35      40   20    0     619   1643   
           3    55   33   44    -12       32      41   21    0     645   1729   
           4    76   56   66     -1       47      56    0    1     644   1745   
           5    76   56   66     -1       37      56    0    1     616   1640   
           6    76   51   64      0       47      55    1    0     644   1745   
           7    58   38   48      0       32      41   17    0     644   1745   
           8    78   55   67      0       32 

In [65]:
train_file = pd.read_csv("Data/train.csv", sep=',')
train_file = train_file[train_file['units'] > 0]

store_station_pairs = match_store_station()
weather_file = read_processed_weather()
train_num = train_file.values[:,1].shape[0]
for ind in tqdm(range(100)):
    store_nbr = train_file.values[:,1][ind]
    date = train_file.values[:,0][ind]  
    weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)
    #weather_data = np.append(train_file.values[:,2:][ind], weather_data, axis = 1) #[item_nbr, units, weather_features]
    weather_data = np.concatenate((train_file.values[:,2:][ind], weather_data), axis = 0)
    date = np.array([date, store_nbr])
    result = np.concatenate((date, weather_data), axis = 0)
    #result = sparse.hstack((date, weather_data))

    item_dataset.append(result)

  0%|          | 0/100 [00:00<?, ?it/s]


NameError: name 'item_dataset' is not defined

In [57]:
def get_weather_data(date, store_nbr, store_station_pairs, weather_file):
    station_nbr = store_station_pairs.loc[store_nbr].values[0]
    weather_data = weather_file.loc[date, station_nbr].values
    return weather_data

In [22]:
def get_item_list():
    #train data에서 item_nbr data중복 제거 후 list로 변환
    train_file = pd.read_csv("Data/train.csv", sep=',')
    item_list = list(set(list(train_file.values[:,2])))
    return item_list

In [23]:
def get_item_data(item_nbr, percent, store_station_pairs, weather_file, train_file):
    start = time.time()
    train_num = train_file.values[:,1].shape[0]

    item_index = np.where(train_file.values[:,2] == item_nbr)
    item_index = item_index[0]
    num = int(item_index.shape[0]*(1-percent)) # Select according to the given percentage, need to be improved
    iter_list = range(num, item_index.shape[0])
    item_dataset = np.zeros((len(iter_list), weather_file.values.shape[1]+1))
    count = 0
    #print len(iter_list)
    for i in iter_list:
        index = item_index[i]
        #print count
        date = train_file.values[:,0][index]
        store_nbr = train_file.values[:,1][index]
        units = train_file.values[:,3][index]

        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)

        item_dataset[count,:] =  np.append(weather_data, [units], axis=0)
        count += 1
    end = time.time()
    #print "Running time for item %d: %f" %(int(item_nbr), (end-start))
    return item_dataset

In [24]:
def write_item_data(tasks):
    item_list = tasks.get()
    #print item_list
    store_station_pairs = match_store_station()
    weather_file = read_processed_weather() # index has 2 columns and values has 17 columns
    train_file = pd.read_csv("Data/train.csv", sep=',')

    #item_list = get_item_list()
    #item_nbr = item_list[0]
    for item_nbr in item_list:
        item_dataset = get_item_data(item_nbr, 0.1, store_station_pairs, weather_file, train_file) # Give percentage
        df = pd.DataFrame(item_dataset, index=None, columns=None)
        filename = "Item_data/item_%d.csv" %(int(item_nbr))
        df.to_csv(filename, sep=',', index=None, columns=None)

In [25]:
def multi_process(traget_func):
    processors = multiprocessing.cpu_count()

    myTasks = multiprocessing.Queue()
    item_list = get_item_list()
    temp_part = []
    div = len(item_list)/(processors-1)
    rem = len(item_list)%(processors-1)
    ind = 0
    while ind < div*(processors-1): 
        temp_part.append(item_list[ind:ind+div])
        ind = ind+div
    temp_part.append(item_list[(len(item_list)-rem):])

    for each in temp_part:
        myTasks.put(each)

    Workers = [multiprocessing.Process(target = traget_func, args =(myTasks,)) for i in range(processors)]

    #Workers[0].start()

    for each in Workers:
        each.start()

In [26]:
def read_item_data(item_nbr):
    filename = "Item_data/item_%d.csv" %(int(item_nbr))
    item_dataset = pd.read_csv(filename, sep=',')

    item_data = item_dataset.values[:,0:-1]
    item_label = item_dataset.values[:,-1]
    return item_data, item_label

In [27]:
def split_item_data(item_nbr, type):
    item_data, item_label = read_item_data(item_nbr)
    X_train, X_test, y_train, y_test = train_test_split(item_data, item_label, test_size=0.25, random_state=42)
    if type == 'train':
        return X_train, y_train
    elif type == 'test':
        return X_test, y_test

In [80]:
def get_full_data():
    train_file = pd.read_csv("Data/train.csv", sep=',')
    train_file = train_file[train_file['units'] > 0]

    store_station_pairs = match_store_station()
    #weather_file = read_weather() # index has 2 columns and values has 18 columns
    weather_file = read_processed_weather()
    train_num = train_file.values[:,1].shape[0]
    item_dataset = [] # Should be 20 columns
    for ind in tqdm(range(train_num)):
        store_nbr = train_file.values[:,1][ind]
        date = train_file.values[:,0][ind]  
        weather_data = get_weather_data(date, store_nbr, store_station_pairs, weather_file)
        #weather_data = np.append(train_file.values[:,2:][ind], weather_data, axis = 1) #[item_nbr, units, weather_features]
        #weather_data = np.concatenate((train_file.values[:,2:][ind], weather_data), axis = 0)
        date = np.array([date, store_nbr])
        result = np.concatenate((date, train_file.values[:,2:][ind], weather_data), axis = 0)
        
        item_dataset.append(result)

    item_dataset = np.array(item_dataset)
    #train_columns = np.append(np.array(['item_nbr','units']), weather_file.columns.values, axis=1)
    train_columns = np.concatenate((np.array(['date', 'store_nbr', 'item_nbr','units']), weather_file.columns.values), axis=0)
    train_df = pd.DataFrame(item_dataset, index=None, columns=train_columns) # Build item_nbr index on units and weather features
    train_df.to_csv("train_feature_matrix.csv", sep=',', index=None)
    return train_df

In [81]:
get_full_data()

100%|██████████| 118696/118696 [2:20:14<00:00, 14.11it/s]  


,date,store_nbr,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2012-01-01,1,9,29,52,31,42,16,36,40,...,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
1,2012-01-01,1,28,2,52,31,42,16,36,40,...,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
2,2012-01-01,1,51,1,52,31,42,16,36,40,...,0,716,1626,0,0.05,29.78,29.92,3.6,20,4.6
3,2012-01-01,2,5,191,50,34,42,5,25,35,...,0,739,1729,0,0,29.13,30.52,11.4,32,11.3
4,2012-01-01,2,44,215,50,34,42,5,25,35,...,0,739,1729,0,0,29.13,30.52,11.4,32,11.3
5,2012-01-01,3,5,214,50,33,42,0,26,35,...,0,727,1742,0,0,29.15,30.54,10.3,32,10.2
6,2012-01-01,3,45,112,50,33,42,0,26,35,...,0,727,1742,0,0,29.15,30.54,10.3,32,10.2
7,2012-01-01,4,9,61,34,19,27,0,17,23,...,0,727,1742,0,0,29.34,30.09,22.8,30,22.5
8,2012-01-01,4,27,21,34,19,27,0,17,23,...,0,727,1742,0,0,29.34,30.09,22.8,30,22.5
9,2012-01-01,5,16,24,72,48,60,7,54,57,...,0,724,1737,0,0,30.15,30.19,4.4,25,4.8


In [6]:
def read_full_data():
    train_features = pd.read_csv("train_feature_matrix.csv", sep=',')
    train_df = pd.DataFrame(train_features.values[:,1:], index=train_features.values[:,0], columns=train_features.columns.values[1:])

    print(train_df)

In [41]:
train_file = pd.read_csv("Data/train.csv", sep=',')
train_file = train_file[train_file['units'] > 0]
train_file = train_file.reset_index()

In [42]:
train_file.tail()

,index,date,store_nbr,item_nbr,units
118691,4617382,2014-10-31,44,5,47
118692,4617445,2014-10-31,44,68,39
118693,4617471,2014-10-31,44,94,1
118694,4617497,2014-10-31,45,9,10
118695,4617514,2014-10-31,45,26,1


In [43]:
train_features = pd.read_csv("train_feature_matrix.csv", sep=',')
train_df = pd.DataFrame(train_features.values[:,1:], index=train_features.values[:,0], columns=train_features.columns.values[1:])
train_df = pd.DataFrame(train_features.values, columns=train_features.columns)
train_df

,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,9.0,29.0,52.0,31.0,42.0,16.0,36.0,40.0,23.0,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
1,28.0,2.0,52.0,31.0,42.0,16.0,36.0,40.0,23.0,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
2,51.0,1.0,52.0,31.0,42.0,16.0,36.0,40.0,23.0,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
3,5.0,191.0,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,739.0,1729.0,0.0,0.00,29.13,30.52,11.4,32.0,11.3
4,44.0,215.0,50.0,34.0,42.0,5.0,25.0,35.0,23.0,0.0,739.0,1729.0,0.0,0.00,29.13,30.52,11.4,32.0,11.3
5,5.0,214.0,50.0,33.0,42.0,0.0,26.0,35.0,23.0,0.0,727.0,1742.0,0.0,0.00,29.15,30.54,10.3,32.0,10.2
6,45.0,112.0,50.0,33.0,42.0,0.0,26.0,35.0,23.0,0.0,727.0,1742.0,0.0,0.00,29.15,30.54,10.3,32.0,10.2
7,9.0,61.0,34.0,19.0,27.0,0.0,17.0,23.0,38.0,0.0,727.0,1742.0,0.0,0.00,29.34,30.09,22.8,30.0,22.5
8,27.0,21.0,34.0,19.0,27.0,0.0,17.0,23.0,38.0,0.0,727.0,1742.0,0.0,0.00,29.34,30.09,22.8,30.0,22.5
9,16.0,24.0,72.0,48.0,60.0,7.0,54.0,57.0,5.0,0.0,724.0,1737.0,0.0,0.00,30.15,30.19,4.4,25.0,4.8


In [53]:
result = pd.merge(train_file, train_df, on=['item_nbr','units'], how='outer')
result


,date,store_nbr,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,wetbulb,...,cool,sunrise,sunset,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,2012-01-01,1,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
1,2012-01-02,31,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
2,2012-01-03,4,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
3,2012-01-04,31,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
4,2012-01-04,45,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
5,2012-01-13,1,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
6,2012-01-23,6,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
7,2012-01-28,6,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
8,2012-02-01,32,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6
9,2012-02-18,45,9,29,52.0,31.0,42.0,16.0,36.0,40.0,...,0.0,716.0,1626.0,0.0,0.05,29.78,29.92,3.6,20.0,4.6


In [40]:
train_file = pd.read_csv("Data/train.csv", sep=',')
train_file = train_file[train_file['units'] > 0]
train_file = train_file[train_file['item_nbr'] == 9]
train_file = train_file[train_file['units'] == 29]
train_file

,date,store_nbr,item_nbr,units
8,2012-01-01,1,9,29
8222,2012-01-02,31,9,29
10109,2012-01-03,4,9,29
17990,2012-01-04,31,9,29
19433,2012-01-04,45,9,29
58616,2012-01-13,1,9,29
108011,2012-01-23,6,9,29
132431,2012-01-28,6,9,29
154853,2012-02-01,32,9,29
239213,2012-02-18,45,9,29
